# Universities and PPP Loans 
#### A Big Local News Project

The Stanford Journalism M.A. cohort partnered with Big Local News, a Stanford project, to analyze Paycheck Protection Payment loan receipt trends among American higher education institutions in 2020.

In [1]:
# Importing libraries
import pandas as pd
import re
import numpy as np
import warnings
import nbconvert

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'
warnings.simplefilter(action='ignore', category=FutureWarning)

# The Data



The project involves four separate databases providing infromation about higher education institutions:


<b>Paycheck Protection Payment data</b>: The PPP data covers loans issued under the Payment Protection Program as part of the $2.2 trillion CARES Act. Data for 2020 is available on Big Local News' online open projects archive at https://biglocalnews.org/#/open_projects. For this project, we used data updated on Feb 2, 2021, by the SBA available at https://sba.app.box.com/s/5myd1nxutoq8wxecx2562baruz774si6. 

<u>Data dictionary</u>: https://www.sba.gov/document/support--7a-504-loan-data-dictionary 


<b> IRS Exempt Organizations Business data</b>: The IRS data provides information about charity and nonprofit organizations from the Internal Revenue Service’s Business Master File. The data can be downloaded from https://www.irs.gov/charities-non-profits/exempt-organizations-business-master-file-extract-eo-bmf.

<u>Data dictionary</u>: https://www.irs.gov/pub/irs-soi/eo_info.pdf


<b> Voluntary Support of Education Survey data</b>: The VSE Survey collects data on fundraising at U.S. higher education institutions. The data is published and distributed annually by CASE to its members or upon request. For more information visit https://www.case.org/resources/voluntary-support-education-survey.


<b> The Integrated Postsecondary Education Data System</b>: IPEDS is a system of surveys conducted annually by the U.S. Department of Education's National Center for Education Statistics (NCES). The annual survey database includes unique identifiers for each institution, and information about staff and salary outlays, among dozens of other variables. The database can be downloaded in Microsoft Access Database format from https://nces.ed.gov/ipeds/use-the-data/download-access-database. 

<hr>

# First Analysis

<b>PPP, IRS and VSE data merge for Higher Ed institutions requesting loans of \\$1M or above</b>: In our first analysis, we merged PPP data for higher education institutions which received loans of $1M or above with the IRS and VSE databases. 

##### 1. LOADING LOANS DATA
- I concatenated BigLocal’s sub 150K files (01 though 05) and plus 150K file and extracted records
- I standardized column names
- I filtered the data to only include higher education institutions (with NAICS code 611310: Colleges, Universities, and Professional Schools) which received 1M or more in loans (536 total)

In [2]:
# creating empty list to store file uploads
loans_files = []

# creating list of file names
loans_file_names = ['public_150k_plus.csv']

# appending uploaded files to files list
for file_name in loans_file_names:
    file = pd.read_csv(file_name, dtype={'Zip':'str', 'OPEID6':'str'})
    file.columns = file.columns.str.lower()
    file = file.rename(columns={'borrowername': 'name_loans',
                               'borrowerstate': 'state'})
    loans_files.append(file)

# concatenating files
loans_merged = pd.concat(loans_files)

# Filtering higher education institutions with loan amount greater or equal to $1M
universities_loans1m = loans_merged[(loans_merged.naicscode == 611310) & (loans_merged.currentapprovalamount >= 1000000)]

display(universities_loans1m.head(3))

,loannumber,dateapproved,sbaofficecode,processingmethod,name_loans,borroweraddress,borrowercity,state,borrowerzip,loanstatusdate,loanstatus,term,sbaguarantypercentage,initialapprovalamount,currentapprovalamount,undisbursedamount,franchisename,servicinglenderlocationid,servicinglendername,servicinglenderaddress,servicinglendercity,servicinglenderstate,servicinglenderzip,ruralurbanindicator,hubzoneindicator,lmiindicator,businessagedescription,projectcity,projectcountyname,projectstate,projectzip,cd,jobsreported,naicscode,raceethnicity,utilities_proceed,payroll_proceed,mortgage_interest_proceed,rent_proceed,refinance_eidl_proceed,health_care_proceed,debt_interest_proceed,businesstype,originatinglenderlocationid,originatinglender,originatinglendercity,originatinglenderstate,gender,veteran,nonprofit
91,8701017007,04/08/2020,1084,PPP,ILISAGVIK COLLEGE,100 Stevenson St,BARROW,AK,99723-0749,NaN,Exemption 4,24,100,2068630.0,1981787.0,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,R,N,NaN,Existing or more than 2 years old,BARROW,NORTH SLOPE,AK,99723-0749,AK-,207.0,611310.0,Unanswered,NaN,1981787.0,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,Y
121,4801407005,04/04/2020,1084,PPP,ALASKA PACIFIC UNIVERSITY,4101 University Dr,ANCHORAGE,AK,99508-4625,01/16/2021,Paid in Full,24,100,1710600.0,1710600.0,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,U,N,NaN,Existing or more than 2 years old,ANCHORAGE,ANCHORAGE,AK,99508-4625,AK-,170.0,611310.0,Unanswered,NaN,1710600.0,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,Y
122,8737768301,01/30/2021,1084,PPS,ALASKA PACIFIC UNIVERSITY,4101 University Dr,Anchorage,AK,99508-4625,01/30/2021,Active Un-Disbursed,60,100,1710600.0,1710600.0,1710600.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,U,N,N,Existing or more than 2 years old,Anchorage,ANCHORAGE,AK,99508-4625,AK-,132.0,611310.0,Unanswered,1.0,1710595.0,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,Y


##### 2. LOADING IRS DATA
- I concatenated the IRS four regional data files - for the northeast area, mid-atlantic and great lakes area, gulf coast and pacific coast area, and other areas. 
- I standardized column names (see chart above for reference)

In [3]:
# creating empty list to store file uploads
irs_files = []

# creating list of file names
irs_file_names = ['eo1.csv', 'eo2.csv', 'eo3.csv', 'eo4.csv']

# appending uploaded files to files list
for file_name in irs_file_names:
    file = pd.read_csv(file_name)
    file.columns = ['ein_code',
                     'name_irs',
                     'ico',
                     'address_irs',
                     'city_irs',
                     'state',
                     'zip_irs',
                     'group',
                     'subsect',
                     'affil',
                     'class',
                     'ruling',
                     'deduct',
                     'foundation',
                     'activity',
                     'org',
                     'status',
                     'tax_period',
                     'asset_code',
                     'income_code',
                     'filing_req_code',
                     'pf_filing_req_code',
                     'accounting_period',
                     'asset_amount',
                     'income_amount',
                     'revenue_amount',
                     'ntee_code',
                     'secondary_name']
    irs_files.append(file)

# concatenating files
irs = pd.concat(irs_files)

display(irs.head(3))

,ein_code,name_irs,ico,address_irs,city_irs,state,zip_irs,group,subsect,affil,class,ruling,deduct,foundation,activity,org,status,tax_period,asset_code,income_code,filing_req_code,pf_filing_req_code,accounting_period,asset_amount,income_amount,revenue_amount,ntee_code,secondary_name
0,19818,PALMER SECOND BAPTIST CHURCH,NaN,1050 THORNDIKE ST,PALMER,MA,01069-1507,3125,3,9,7000,195504,1,10,1000000,5,1,NaN,0,0,6,0,12,NaN,NaN,NaN,NaN,3514
1,29215,ST GEORGE CATHEDRAL,NaN,523 E BROADWAY,SOUTH BOSTON,MA,02127-4415,2365,3,9,7000,197210,1,10,1000000,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN
2,587764,IGLESIA BETHESDA INC,NaN,157 ANDOVER ST,LOWELL,MA,01852-2348,0,3,3,7000,200401,1,10,0,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,NaN


##### 3. MERGING LOANS AND IRS DATA

<b>First Merge</b>
- I created a standard_name column in each file based on the name column (removed punctuation, common short words, corporate abbreviations and references, initials and single letters, etc.).
- I merged the dataframes on the standard name and state columns
- I created a separate matched_first dataframe

In [4]:
# Creating list with words to remove
remove_words = ['INC', 'LLP', 'LLC', 'FOUNDATION', 'INC', 'INCORPORATED', 'CORP', 'CORPORATION', 'CO', 'BOARD', 'TRUSTEES', 'OF', 'AND', 'THE']

# Defining standardizing function
def standard_name(df, name_col):
    df['standard_name'] = df[name_col].str.replace('[^\w\s]','').str.upper() 
    df['standard_name'] = df['standard_name'].apply(lambda x: ' '.join([word for word in x.split() if word not in (remove_words) and len(word) > 1])).str.strip()
    df['standard_name'] = df['standard_name'].apply(lambda x: re.sub(r"\\SO\b", "SOUTH", x))
    df['standard_name'] = df['standard_name'].apply(lambda x: re.sub(r"\\CAL\b", "CALIFORNIA", x))
    return df['standard_name']

# Applying function to df
irs['standard_name'] = standard_name(irs, 'name_irs')
universities_loans1m['standard_name'] = standard_name(universities_loans1m, 'name_loans')

# Merging dfs
merged_loans_irs_first = pd.merge(universities_loans1m, irs, on=['standard_name', 'state'], how='left')

# Creating df of matched records
matched_first = merged_loans_irs_first[merged_loans_irs_first.name_irs.isnull() == False]

print('\nMatched loan - irs records:', len(merged_loans_irs_first[merged_loans_irs_first.ein_code.isnull() == False]))
print('Unmatched loan - irs records:', len(merged_loans_irs_first[merged_loans_irs_first.ein_code.isnull() == True]))

display(merged_loans_irs_first.head(3))

merged_loans_irs_first.to_csv('merged_loans_irs_first.csv')


Matched loan - irs records: 508
Unmatched loan - irs records: 101


,loannumber,dateapproved,sbaofficecode,processingmethod,name_loans,borroweraddress,borrowercity,state,borrowerzip,loanstatusdate,loanstatus,term,sbaguarantypercentage,initialapprovalamount,currentapprovalamount,undisbursedamount,franchisename,servicinglenderlocationid,servicinglendername,servicinglenderaddress,servicinglendercity,servicinglenderstate,servicinglenderzip,ruralurbanindicator,hubzoneindicator,lmiindicator,businessagedescription,projectcity,projectcountyname,projectstate,projectzip,cd,jobsreported,naicscode,raceethnicity,utilities_proceed,payroll_proceed,mortgage_interest_proceed,rent_proceed,refinance_eidl_proceed,health_care_proceed,debt_interest_proceed,businesstype,originatinglenderlocationid,originatinglender,originatinglendercity,originatinglenderstate,gender,veteran,nonprofit,standard_name,ein_code,name_irs,ico,address_irs,city_irs,zip_irs,group,subsect,affil,class,ruling,deduct,foundation,activity,org,status,tax_period,asset_code,income_code,filing_req_code,pf_filing_req_code,accounting_period,asset_amount,income_amount,revenue_amount,ntee_code,secondary_name
0,8701017007,04/08/2020,1084,PPP,ILISAGVIK COLLEGE,100 Stevenson St,BARROW,AK,99723-0749,NaN,Exemption 4,24,100,2068630.0,1981787.0,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,R,N,NaN,Existing or more than 2 years old,BARROW,NORTH SLOPE,AK,99723-0749,AK-,207.0,611310.0,Unanswered,NaN,1981787.0,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,Y,ILISAGVIK COLLEGE,201650054.0,ILISAGVIK COLLEGE FOUNDATION,% EDNA AHGEAK MACLEAN,PO BOX 749,BARROW,99723-0749,0.0,3.0,3.0,1200.0,201310.0,1.0,21.0,0.0,1.0,1.0,201906.0,7.0,6.0,1.0,0.0,6.0,7522675.0,2920125.0,647065.0,B82,NaN
1,8701017007,04/08/2020,1084,PPP,ILISAGVIK COLLEGE,100 Stevenson St,BARROW,AK,99723-0749,NaN,Exemption 4,24,100,2068630.0,1981787.0,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,R,N,NaN,Existing or more than 2 years old,BARROW,NORTH SLOPE,AK,99723-0749,AK-,207.0,611310.0,Unanswered,NaN,1981787.0,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,Y,ILISAGVIK COLLEGE,920158414.0,ILISAGVIK COLLEGE,% ACCOUNTING TECH PAYROLL TRAVEL,PO BOX 749,BARROW,99723-0749,0.0,3.0,3.0,2000.0,199612.0,1.0,11.0,320000000.0,1.0,1.0,201906.0,8.0,8.0,1.0,0.0,6.0,13032436.0,17114732.0,17114732.0,B41Z,NaN
2,4801407005,04/04/2020,1084,PPP,ALASKA PACIFIC UNIVERSITY,4101 University Dr,ANCHORAGE,AK,99508-4625,01/16/2021,Paid in Full,24,100,1710600.0,1710600.0,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,U,N,NaN,Existing or more than 2 years old,ANCHORAGE,ANCHORAGE,AK,99508-4625,AK-,170.0,611310.0,Unanswered,NaN,1710600.0,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,Y,ALASKA PACIFIC UNIVERSITY,920023588.0,ALASKA PACIFIC UNIVERSITY,% SHEILA KING,4101 UNIVERSITY DR,ANCHORAGE,99508-4625,0.0,3.0,3.0,2000.0,196203.0,1.0,11.0,46030029.0,1.0,1.0,201906.0,9.0,8.0,1.0,0.0,6.0,134770382.0,44406826.0,17589339.0,B430,NaN


##### ** MANUALLY ENTERED NAME_IRS IN UNMATCHED FIRST RECORDS - PROPUBLICA SEARCH ENGINE: https://projects.propublica.org/nonprofits/organizations/610444763 **

<b>Second Merge</b>
- I uploaded the manually revised dataframe and stored it as unmatched_first
- I removed accidental spaces from the manually revised name_irs column in the dataframe
- I merged the dataframes on the name_irs and state columns

In [5]:
# loading manually revised file
unmatched_first = pd.read_csv('merged_loans_irs_edited.csv')

# stripping name_irs column
unmatched_first.name_irs = unmatched_first.name_irs.str.strip()

# Merging dfs
merged_loans_irs_second = pd.merge(unmatched_first, irs, on=['name_irs', 'state'], how='left')

# Concatenating matched in first merge and all in second merge
merged_loans_irs = pd.concat([matched_first, merged_loans_irs_second])

display(merged_loans_irs.head(3))

,loannumber,dateapproved,sbaofficecode,processingmethod,name_loans,borroweraddress,borrowercity,state,borrowerzip,loanstatusdate,loanstatus,term,sbaguarantypercentage,initialapprovalamount,currentapprovalamount,undisbursedamount,franchisename,servicinglenderlocationid,servicinglendername,servicinglenderaddress,servicinglendercity,servicinglenderstate,servicinglenderzip,ruralurbanindicator,hubzoneindicator,lmiindicator,businessagedescription,projectcity,projectcountyname,projectstate,projectzip,cd,jobsreported,naicscode,raceethnicity,utilities_proceed,payroll_proceed,mortgage_interest_proceed,rent_proceed,refinance_eidl_proceed,health_care_proceed,debt_interest_proceed,businesstype,originatinglenderlocationid,originatinglender,originatinglendercity,originatinglenderstate,gender,veteran,nonprofit,standard_name,ein_code,name_irs,ico,address_irs,city_irs,zip_irs,group,subsect,affil,class,ruling,deduct,foundation,activity,org,status,tax_period,asset_code,income_code,filing_req_code,pf_filing_req_code,accounting_period,asset_amount,income_amount,revenue_amount,ntee_code,secondary_name,Unnamed: 0
0,8701017007,04/08/2020,1084,PPP,ILISAGVIK COLLEGE,100 Stevenson St,BARROW,AK,99723-0749,NaN,Exemption 4,24,100,2068630.0,1981787.0,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,R,N,NaN,Existing or more than 2 years old,BARROW,NORTH SLOPE,AK,99723-0749,AK-,207.0,611310.0,Unanswered,NaN,1981787.0,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,Y,ILISAGVIK COLLEGE,201650054.0,ILISAGVIK COLLEGE FOUNDATION,% EDNA AHGEAK MACLEAN,PO BOX 749,BARROW,99723-0749,0.0,3.0,3.0,1200.0,201310.0,1.0,21.0,0.0,1.0,1.0,201906.0,7.0,6.0,1.0,0.0,6.0,7522675.0,2920125.0,647065.0,B82,NaN,NaN
1,8701017007,04/08/2020,1084,PPP,ILISAGVIK COLLEGE,100 Stevenson St,BARROW,AK,99723-0749,NaN,Exemption 4,24,100,2068630.0,1981787.0,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,R,N,NaN,Existing or more than 2 years old,BARROW,NORTH SLOPE,AK,99723-0749,AK-,207.0,611310.0,Unanswered,NaN,1981787.0,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,Y,ILISAGVIK COLLEGE,920158414.0,ILISAGVIK COLLEGE,% ACCOUNTING TECH PAYROLL TRAVEL,PO BOX 749,BARROW,99723-0749,0.0,3.0,3.0,2000.0,199612.0,1.0,11.0,320000000.0,1.0,1.0,201906.0,8.0,8.0,1.0,0.0,6.0,13032436.0,17114732.0,17114732.0,B41Z,NaN,NaN
2,4801407005,04/04/2020,1084,PPP,ALASKA PACIFIC UNIVERSITY,4101 University Dr,ANCHORAGE,AK,99508-4625,01/16/2021,Paid in Full,24,100,1710600.0,1710600.0,0.0,NaN,116975,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,U,N,NaN,Existing or more than 2 years old,ANCHORAGE,ANCHORAGE,AK,99508-4625,AK-,170.0,611310.0,Unanswered,NaN,1710600.0,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,116975,Northrim Bank,ANCHORAGE,AK,Unanswered,Unanswered,Y,ALASKA PACIFIC UNIVERSITY,920023588.0,ALASKA PACIFIC UNIVERSITY,% SHEILA KING,4101 UNIVERSITY DR,ANCHORAGE,99508-4625,0.0,3.0,3.0,2000.0,196203.0,1.0,11.0,46030029.0,1.0,1.0,201906.0,9.0,8.0,1.0,0.0,6.0,134770382.0,44406826.0,17589339.0,B430,NaN,NaN


In [6]:
print('\nRecords in ProPublica database but not IRS database\n')
display(merged_loans_irs_second[(merged_loans_irs_second.name_irs.isnull() == False) & (merged_loans_irs_second.ein_code.isnull() == True)])


Records in ProPublica database but not IRS database



,Unnamed: 0,loannumber,dateapproved,sbaofficecode,processingmethod,name_loans,borroweraddress,borrowercity,state,borrowerzip,loanstatusdate,loanstatus,term,sbaguarantypercentage,initialapprovalamount,currentapprovalamount,undisbursedamount,franchisename,servicinglenderlocationid,servicinglendername,servicinglenderaddress,servicinglendercity,servicinglenderstate,servicinglenderzip,ruralurbanindicator,hubzoneindicator,lmiindicator,businessagedescription,projectcity,projectcountyname,projectstate,projectzip,cd,jobsreported,naicscode,raceethnicity,utilities_proceed,payroll_proceed,mortgage_interest_proceed,rent_proceed,refinance_eidl_proceed,health_care_proceed,debt_interest_proceed,businesstype,originatinglenderlocationid,originatinglender,originatinglendercity,originatinglenderstate,gender,veteran,nonprofit,name_irs,ein_code,ico,address_irs,city_irs,zip_irs,group,subsect,affil,class,ruling,deduct,foundation,activity,org,status,tax_period,asset_code,income_code,filing_req_code,pf_filing_req_code,accounting_period,asset_amount,income_amount,revenue_amount,ntee_code,secondary_name,standard_name
7,101,6157407100,04/14/2020,455,PPP,AVE MARIE UNIVERSITY. INC,5050 Ave Maria Blvd,IMMOKALEE,FL,34142,NaN,Exemption 4,24,100,2700197.00,2700197.00,0.0,NaN,451106,First Florida Integrity Bank,3560 Kraft Rd,NAPLES,FL,34105-5035,U,N,NaN,Existing or more than 2 years old,IMMOKALEE,COLLIER,FL,34142-1101,FL-25,266.0,611310.0,Unanswered,200197.0,2400000.00,NaN,NaN,NaN,100000.0,NaN,Non-Profit Organization,451106,First Florida Integrity Bank,NAPLES,FL,Unanswered,Unanswered,Y,AVE MARIA UNIVERSITY SUPPORTING TRUST INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,219,5982497100,04/14/2020,457,PPP,"LINDSEY WILSON COLLEGE, INC",210 LINDSEY WILSON STREET,COLUMBIA,KY,42728,NaN,Exemption 4,24,100,4201600.00,4201600.00,0.0,NaN,27783,"Community Trust Bank, Inc.",346 N Mayo Trl,PIKEVILLE,KY,41501-1847,R,Y,NaN,Existing or more than 2 years old,COLUMBIA,ADAIR,KY,42728-0001,KY-01,459.0,611310.0,Unanswered,NaN,4201600.00,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,27783,"Community Trust Bank, Inc.",PIKEVILLE,KY,Unanswered,Unanswered,Y,LINDSEY WILSON COLLEGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,105,1708087301,04/28/2020,455,PPP,THE COLLEGE OF THE FLORIDA KEYS,5901 College Rd,KEY WEST,FL,33040,NaN,Exemption 4,24,100,1742208.43,1742208.43,0.0,NaN,4392,Centennial Bank,620 Chestnut St,CONWAY,AR,72032-5404,U,N,NaN,Existing or more than 2 years old,KEY WEST,MONROE,FL,33040-2300,FL-26,331.0,611310.0,Unanswered,NaN,1742208.43,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,4392,Centennial Bank,CONWAY,AR,Unanswered,Unanswered,Y,FLORIDA KEYS COLLEGE CAMPUS FOUNDATION INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 4. LOADING VSE DATA
- I concatenated the VSE files for years 2009 through 2020
- I standardized column names (see chart above for reference)

In [7]:
# creating empty list to store file uploads
vse_files = []

# creating list of file names
vse_file_names = ['vse_1920.csv', 'vse_1718.csv', 'vse_1516.csv', 'vse_1314.csv', 'vse_1112.csv', 'vse_0910.csv']

# appending uploaded files to files list
for file_name in vse_file_names:
    file = pd.read_csv(file_name)
    file.columns = ['year_vse',
                    'data_group',
                    'case_id',
                    'state',
                    'ownership',
                    'ipeds_code',
                    'country',
                    'found_year',
                    'class_18',
                    'class_15',
                    'name_vse',
                    'all_curr_ops_unr',
                    'all_curr_ops_r',
                    'all_curr_ops',
                    'all_cap_purp_pbe',
                    'all_cap_purp_end_unr',
                    'all_cap_purp_loan',
                    'all_cap_purp_end_r',
                    'all_cap_purp_gifts',
                    'all_deferred_face',
                    'all_deferred_present',
                    'all_fundraising',
                    'rel_curr_ops_unr',
                    'rel_curr_ops_r',
                    'rel_curr_ops',
                    'rel_cap_purp_pbe',
                    'rel_cap_purp_unr',
                    'rel_cap_purp_r',
                    'rel_cap_purp_loan',
                    'rel_cap_purp',
                    'rel_fundraising']

    vse_files.append(file)

# concatenating files
vse = pd.concat(vse_files)

# applying standardizing function
vse['standard_name'] = standard_name(vse, 'name_vse')

display(vse.head(3))

,year_vse,data_group,case_id,state,ownership,ipeds_code,country,found_year,class_18,class_15,name_vse,all_curr_ops_unr,all_curr_ops_r,all_curr_ops,all_cap_purp_pbe,all_cap_purp_end_unr,all_cap_purp_loan,all_cap_purp_end_r,all_cap_purp_gifts,all_deferred_face,all_deferred_present,all_fundraising,rel_curr_ops_unr,rel_curr_ops_r,rel_curr_ops,rel_cap_purp_pbe,rel_cap_purp_unr,rel_cap_purp_r,rel_cap_purp_loan,rel_cap_purp,rel_fundraising,standard_name
0,FY2020,All U.S. Higher Education (with 2015 Carnegie),17834,KY,Public,247940.0,United States,1986.0,Associate's Colleges: Mixed Transfer/Career & ...,Associate's: High Career & Technical-Mixed Tra...,Owensboro Community and Technical College,"$122,502","$141,751","$264,253","$10,500",$0,$0,"$27,200","$37,700",$0,$0,"$301,953",$600,$0,$600,$0,$0,$0,$0,$0,$600,OWENSBORO COMMUNITY TECHNICAL COLLEGE
1,FY2019,All U.S. Higher Education (with 2015 Carnegie),17834,KY,Public,247940.0,United States,1986.0,Associate's Colleges: Mixed Transfer/Career & ...,Associate's: High Career & Technical-Mixed Tra...,Owensboro Community and Technical College,"$167,778","$207,562","$375,340","$53,805",$0,$0,"$85,350","$139,155",$0,$0,"$514,495","$3,200",$0,"$3,200",$0,$0,$0,$0,$0,"$3,200",OWENSBORO COMMUNITY TECHNICAL COLLEGE
2,FY2020,All U.S. Higher Education (with 2015 Carnegie),17847,AL,Public,100812.0,United States,1822.0,Baccalaureate Colleges: Diverse Fields,Baccalaureate: Diverse Fields,Athens State University,"$66,457","$184,048","$250,505","$3,027","$1,200",$0,"$338,597","$342,824",NaN,$0,"$593,329",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$0,ATHENS STATE UNIVERSITY


##### 5. MERGING LOANS-IRS AND VSE DATA
- I filtered the 2020 vse data and merged it with the loans-irs file created in Step 3. 
- I separated matched and unmatched records from this merge into matched_20 and unmatched_20
- I remerged unmatched_20 records with 2019 data. 
- I continued with this process for 2018 through 2009. 

This process was meant to avoid duplicates and only include the most recent VSE data for each university. 

In [8]:
def merge_unmatched(year, unmatched):
    
    vse_year = vse[vse.year_vse == 'FY' + year]
    merged_loans_vse = pd.merge(unmatched, vse_year, on=['state', 'standard_name'], how='left')
    matched = merged_loans_vse[merged_loans_vse.year_vse.isnull() == False]
    unmatched = merged_loans_vse[merged_loans_vse.year_vse.isnull() == True].iloc[:, : len(unmatched.columns)]
    
    return matched, unmatched

matched_20 = merge_unmatched('2020', merged_loans_irs)[0]
unmatched_20 = merge_unmatched('2020', merged_loans_irs)[1]

matched_19 = merge_unmatched('2019', unmatched_20)[0]
unmatched_19 = merge_unmatched('2019', unmatched_20)[1]

matched_18 = merge_unmatched('2018', unmatched_19)[0]
unmatched_18 = merge_unmatched('2018', unmatched_19)[1]

matched_17 = merge_unmatched('2017', unmatched_18)[0]
unmatched_17 = merge_unmatched('2017', unmatched_18)[1]

matched_16 = merge_unmatched('2016', unmatched_17)[0]
unmatched_16 = merge_unmatched('2016', unmatched_17)[1]

matched_15 = merge_unmatched('2015', unmatched_16)[0]
unmatched_15 = merge_unmatched('2015', unmatched_16)[1]

matched_14 = merge_unmatched('2014', unmatched_15)[0]
unmatched_14 = merge_unmatched('2014', unmatched_15)[1]

matched_13 = merge_unmatched('2013', unmatched_14)[0]
unmatched_13 = merge_unmatched('2013', unmatched_14)[1]

matched_12 = merge_unmatched('2012', unmatched_13)[0]
unmatched_12 = merge_unmatched('2012', unmatched_13)[1]

matched_11 = merge_unmatched('2011', unmatched_12)[0]
unmatched_11 = merge_unmatched('2011', unmatched_12)[1]

matched_10 = merge_unmatched('2010', unmatched_11)[0]
unmatched_10 = merge_unmatched('2010', unmatched_11)[1]

matched_09 = merge_unmatched('2009', unmatched_10)[0]
unmatched_09 = merge_unmatched('2009', unmatched_10)[1]

merged_loans_irs_vse = pd.concat([matched_20, matched_19, matched_18, matched_17, matched_16, matched_15, matched_14, matched_13, matched_12, matched_11, matched_10, matched_09, unmatched_09])

In [9]:
print('\nMatched loan - irs - vse records:', len(merged_loans_irs_vse[merged_loans_irs_vse.year_vse.isnull() == False]))
print('Unmatched loan - irs - vse records:', len(merged_loans_irs_vse[merged_loans_irs_vse.year_vse.isnull() == True]))


Matched loan - irs - vse records: 184
Unmatched loan - irs - vse records: 414


##### 6. EXPORTING CSV

In [10]:
merged_loans_irs_vse.to_csv('merged_loans_irs_vse.csv')

<hr>

# Second Analysis

<b>PPP and IPEDS data merge for all Higher Ed institutions</b>: In our second analysis, we merged PPP data and select IPEDS surveys to compare each university's loan amount to the size of its staff and its annual salary outlays. 

##### 1. LOADING LOANS DATA
- I filtered the PPP data uploaded in step 1 of the first analysis to only include higher education institutions (with NAICS code 611310: Colleges, Universities, and Professional Schools). For this analysis, we include all institutions regardless of loan amount.
- I created a standard name column with the standardizing function from the first analysis
- I grouped the data to avoid duplicates

In [11]:
# Filtering higher education institutions
universities_loans = loans_merged[(loans_merged.naicscode == 611310)]

# Creating standard name col
universities_loans['standard_name'] = standard_name(universities_loans, 'name_loans')

# Grouping by standard name and state
universities_loans = universities_loans.groupby(by=['standard_name', 'state']).agg({
    'borroweraddress': 'min',
    'borrowercity': 'min',
    'borrowerzip': 'min',
    'dateapproved': list,
    'currentapprovalamount': 'sum',
    'utilities_proceed': 'sum',
    'payroll_proceed': 'sum',
    'mortgage_interest_proceed': 'sum',
    'rent_proceed': 'sum',
    'refinance_eidl_proceed': 'sum',
    'health_care_proceed': 'sum',
    'debt_interest_proceed': 'sum',
    'jobsreported': 'max'
}).reset_index()

universities_loans.head(5)

,standard_name,state,borroweraddress,borrowercity,borrowerzip,dateapproved,currentapprovalamount,utilities_proceed,payroll_proceed,mortgage_interest_proceed,rent_proceed,refinance_eidl_proceed,health_care_proceed,debt_interest_proceed,jobsreported
0,1440 OPCO,CA,800 Bethany Dr,SCOTTS VALLEY,95066,[04/16/2020],1970875.0,246359.0,1478157.0,0.0,246359.0,0.0,0.0,0.0,237.0
1,ABRAHAM LINCOLN UNIVERSITY,CA,100 WEST BROADWAY SUITE 600,GLENDALE,91210,[05/04/2020],209482.0,0.0,209482.0,0.0,0.0,0.0,0.0,0.0,39.0
2,ACADEMY CHINESE CULTURE HEALTH SCIENCES,CA,"1600 BROADWAY, STE 200",OAKLAND,94612-2100,"[01/20/2021, 05/02/2020]",416892.0,1.0,416890.0,0.0,0.0,0.0,0.0,0.0,105.0
3,ACADEMY FOR ADVANCEMENT CHILDREN WITH AUTISM A...,CA,6742 E. AVENUE H,LANCASTER,93535-7849,[04/27/2020],442000.0,0.0,442000.0,0.0,0.0,0.0,0.0,0.0,75.0
4,ACADEMY FOR JEWISH RELIGION CALIFORNIA,CA,441 s. barrington ave 201,"los angeles,",90049-6468,[04/30/2020],170000.0,0.0,170000.0,0.0,0.0,0.0,0.0,0.0,25.0


##### 2. LOADING IPEDS DATA
I loaded three tables from IPEDS:
- HD2019: 2019 administrative data including institution names, unique identifiers and geographic information
- SAL2019_IS: 2019 data for instructional staff including counts and salary
- SAL2019-NIS: 2019 data for non-instructional staff including counts and salary

In [12]:
# Loading IPEDS data
ipeds_names = pd.read_csv('HD2019.csv', dtype={'UNITID':'str'}) # administrative data (with unique identifiers)
ipeds_is = pd.read_csv('SAL2019_IS.csv', dtype={'UNITID':'str'}) # instructional staff data
ipeds_nis = pd.read_csv('SAL2019_NIS.csv', dtype={'UNITID':'str'}) # non-instructional staff data

##### 3. CLEANING AND GROUPING IPEDS DATA
- I grouped the instructional and non-instructional staff data files to obtain staff count and salary outlay totals for each unique institution
- I then merged the staff grouped data with the administrative "ipeds_names" dataframe on the unique identifier column
- As I did for the loans data, I created a standard name column. Then, I renamed the state column.

In [13]:
# Grouping instructional staff data by unique identifier
is_grouped = ipeds_is.groupby(by='UNITID')[['SAINSTT', 'SAOUTLT']].sum().reset_index()
is_grouped.head(2)

,UNITID,SAINSTT,SAOUTLT
0,100654,484,33441882
1,100663,1812,210689910


In [14]:
# Grouping non-instructional staff data by unique identifier
nis_grouped = ipeds_nis.groupby(by='UNITID')[['SANIN01', 'SANIT01']].sum().reset_index()
nis_grouped.head(2)

,UNITID,SANIN01,SANIT01
0,100654,483,26702523.0
1,100663,3392,205899877.0


In [15]:
# Merging grouped instructional and non-instructional staff data on unique identifier
ipeds_staff = pd.merge(is_grouped, nis_grouped, on='UNITID')
ipeds_staff.head(2)

,UNITID,SAINSTT,SAOUTLT,SANIN01,SANIT01
0,100654,484,33441882,483,26702523.0
1,100663,1812,210689910,3392,205899877.0


In [16]:
# Merging staff and administrative data on unique identifier
ipeds_staff_names = pd.merge(ipeds_staff, ipeds_names, on='UNITID')
ipeds_staff_names.head(2)

# Applying name standardizing function to df and renaming state col
ipeds_staff_names['standard_name'] = standard_name(ipeds_staff_names, 'INSTNM')
ipeds_staff_names = ipeds_staff_names.rename(columns={'STABBR': 'state'})

##### 4. MERGING LOANS AND IPEDS DATA

<b>First Merge</b>
- I merged the dataframes on the standard name and state columns
- I separated matched and unmatched records by storing them in separate dataframes

In [17]:
# Merging loans and ipeds dfs on standard name and state
ipeds_loans = pd.merge(universities_loans, ipeds_staff_names, on=['standard_name', 'state'], how='left')

display(ipeds_loans.head(2))

print('NULL VALUES:')
display(len(ipeds_loans[ipeds_loans.UNITID.isnull() == True]))

print('VALID VALUES:')
display(len(ipeds_loans[ipeds_loans.UNITID.isnull() == False]))

,standard_name,state,borroweraddress,borrowercity,borrowerzip,dateapproved,currentapprovalamount,utilities_proceed,payroll_proceed,mortgage_interest_proceed,rent_proceed,refinance_eidl_proceed,health_care_proceed,debt_interest_proceed,jobsreported,UNITID,SAINSTT,SAOUTLT,SANIN01,SANIT01,INSTNM,IALIAS,ADDR,CITY,ZIP,FIPS,OBEREG,CHFNM,CHFTITLE,GENTELE,EIN,DUNS,OPEID,OPEFLAG,WEBADDR,ADMINURL,FAIDURL,APPLURL,NPRICURL,VETURL,ATHURL,DISAURL,SECTOR,ICLEVEL,CONTROL,HLOFFER,UGOFFER,GROFFER,HDEGOFR1,DEGGRANT,HBCU,HOSPITAL,MEDICAL,TRIBAL,LOCALE,OPENPUBL,ACT,NEWID,DEATHYR,CLOSEDAT,CYACTIVE,POSTSEC,PSEFLAG,PSET4FLG,RPTMTH,INSTCAT,C18BASIC,C18IPUG,C18IPGRD,C18UGPRF,C18ENPRF,C18SZSET,C15BASIC,CCBASIC,CARNEGIE,LANDGRNT,INSTSIZE,F1SYSTYP,F1SYSNAM,F1SYSCOD,CBSA,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,1440 OPCO,CA,800 Bethany Dr,SCOTTS VALLEY,95066,[04/16/2020],1970875.0,246359.0,1478157.0,0.0,246359.0,0.0,0.0,0.0,237.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABRAHAM LINCOLN UNIVERSITY,CA,100 WEST BROADWAY SUITE 600,GLENDALE,91210,[05/04/2020],209482.0,0.0,209482.0,0.0,0.0,0.0,0.0,0.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


NULL VALUES:


467

VALID VALUES:


578

In [18]:
# Creating matched and unmatched dfs
ipeds_loans_matched = ipeds_loans[ipeds_loans.UNITID.isnull() == False]
ipeds_loans_unmatched = ipeds_loans[ipeds_loans.UNITID.isnull() == True]

##### ** MANUALLY ENTERED UNITID IN UNMATCHED DATAFRAME - NCES UNIVERSITY SEARCH ENGINE: https://nces.ed.gov/globallocator/index.asp **

<b>Second Merge</b>
- I uploaded the manually revised dataframe and stored it
- I dropped null rows
- I merged the dataframes on the unique identifier and state columns
- I then concatenated the files resulting from my first and second merges

In [19]:
# Loading edited df
ipeds_loans_unmatched_edited = pd.read_csv('ipeds_loans_unmatched_edited.csv', dtype={'UNITID':'str'}).dropna(axis=1, how='all')

print('NULL VALUES:')
display(len(ipeds_loans_unmatched_edited[ipeds_loans_unmatched_edited.UNITID.isnull() == True]))

print('VALID VALUES:')
display(len(ipeds_loans_unmatched_edited[ipeds_loans_unmatched_edited.UNITID.isnull() == False]))

NULL VALUES:


271

VALID VALUES:


189

In [20]:
# Dropping null rows
ipeds_loans_unmatched_edited = ipeds_loans_unmatched_edited.dropna(subset=['UNITID'])

# Merging loans and ipeds dfs on unique identifier and state
ipeds_loans_unmatched_merged = pd.merge(
    ipeds_loans_unmatched_edited, 
    ipeds_staff_names.drop(columns='standard_name'), 
    on=['UNITID', 'state'], how='left')

display(ipeds_loans_unmatched_merged.head(2))

,Unnamed: 0,standard_name,state,borroweraddress,borrowercity,borrowerzip,dateapproved,currentapprovalamount,utilities_proceed,payroll_proceed,mortgage_interest_proceed,rent_proceed,refinance_eidl_proceed,health_care_proceed,debt_interest_proceed,UNITID,Unnamed: 0.1,SAINSTT,SAOUTLT,SANIN01,SANIT01,INSTNM,IALIAS,ADDR,CITY,ZIP,FIPS,OBEREG,CHFNM,CHFTITLE,GENTELE,EIN,DUNS,OPEID,OPEFLAG,WEBADDR,ADMINURL,FAIDURL,APPLURL,NPRICURL,VETURL,ATHURL,DISAURL,SECTOR,ICLEVEL,CONTROL,HLOFFER,UGOFFER,GROFFER,HDEGOFR1,DEGGRANT,HBCU,HOSPITAL,MEDICAL,TRIBAL,LOCALE,OPENPUBL,ACT,NEWID,DEATHYR,CLOSEDAT,CYACTIVE,POSTSEC,PSEFLAG,PSET4FLG,RPTMTH,INSTCAT,C18BASIC,C18IPUG,C18IPGRD,C18UGPRF,C18ENPRF,C18SZSET,C15BASIC,CCBASIC,CARNEGIE,LANDGRNT,INSTSIZE,F1SYSTYP,F1SYSNAM,F1SYSCOD,CBSA,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,135,ECCLESIA,AR,9653 Nations Drive,Springdale,72762-8159,['04/07/2020'],217180.0,10859.0,162885.0,21718.0,21718.0,0,0.0,0,446233,NaN,30.0,265830.0,28.0,719190.0,Ecclesia College,,9653 Nations Dr,Springdale,72762,5.0,5.0,Dr. Randall Bell,President,4792487236,710477536.0,084936806,3855300.0,1.0,ecollege.edu/,ecollege.edu/admissions/,ecollege.edu/financial-aid/,ecollege.edu/apply/,ecollege.edu/financial-aid/cost-of-education/,,,ecollege.edu/academics/,2.0,1.0,2.0,7.0,1.0,1.0,20.0,1.0,2.0,2.0,2.0,2.0,41.0,1.0,A,-2.0,-2.0,-2,1.0,1.0,1.0,1.0,1.0,2.0,24.0,19.0,3.0,13.0,2.0,8.0,24.0,24.0,-2.0,2.0,1.0,1.0,Ecclesia Inc. ...,200540.0,22220.0,1.0,-2.0,-2.0,5007.0,Benton County,503.0,-94.236643,36.214100,151.0,2.0
1,378,WILLIAMS BAPTIST COLLEGE,AR,60 W Fulbright Ave,WALNUT RIDGE,72476-9000,['04/04/2020'],959500.0,0.0,719625.0,0.0,0.0,0,0.0,239875,107877,NaN,60.0,2952858.0,75.0,2622692.0,Williams Baptist University,,60 W Fulbright Avenue,Walnut Ridge,72476,5.0,5.0,Dr. Stan Norman,President,8708866741,710240677.0,077397917,110600.0,1.0,https://williamsbaptistuniversity.com/,https://williamsbu.edu/admission-requirements-...,https://williamsbu.edu/everything-financial/,https://williamsbu.edu/apply/,https://npc.collegeboard.org/app/wbcoll,,https://williamsbu.edu/wp-content/uploads/2018...,https://williamsbu.edu/disability-services/,2.0,1.0,2.0,7.0,1.0,1.0,20.0,1.0,2.0,2.0,2.0,2.0,42.0,1.0,A,-2.0,-2.0,-2,1.0,1.0,1.0,1.0,1.0,2.0,21.0,9.0,0.0,13.0,3.0,8.0,22.0,22.0,32.0,2.0,1.0,2.0,-2 ...,-2.0,-2.0,-2.0,-2.0,-2.0,5075.0,Lawrence County,501.0,-90.940277,36.125676,138.0,2.0


In [21]:
# Concatenating merged dfs and removing null cols
ipeds_loans_cp = pd.concat([ipeds_loans_matched, ipeds_loans_unmatched_merged])
ipeds_loans_cp = ipeds_loans_cp.loc[:, ~ipeds_loans_cp.columns.str.startswith('Unnamed')]
ipeds_loans_cp = ipeds_loans_cp[ipeds_loans_cp.INSTNM.isnull() == False]

print('NULL VALUES:')
display(len(ipeds_loans_cp[ipeds_loans_cp.INSTNM.isnull() == True]))

print('VALID VALUES:')
display(len(ipeds_loans_cp[ipeds_loans_cp.INSTNM.isnull() == False]))

NULL VALUES:


0

VALID VALUES:


720

##### 5. CALCULATING RATIOS, PERCENTAGES AND SUMMARY TABLES
- I calculated staff count and staff salary outlay totals. Then, I calculated loan-to-staff ratio and loan-to-salary outlays percentage, and their medians across the full dataframe.
- I summarized my calculations in a summary table which includes descriptive information for each university

In [22]:
# Creating calculated cols
ipeds_loans_cp['staff_count'] = ipeds_loans_cp.SAINSTT + ipeds_loans_cp.SANIN01 # instr and non-instr staff
ipeds_loans_cp['staff_salary_outlays'] = ipeds_loans_cp.SAOUTLT + ipeds_loans_cp.SANIT01 

# Creating summary table
summary_table = ipeds_loans_cp[[
                    'UNITID',
                    'INSTNM',
                    'ADDR',
                    'CITY',
                    'state',
                    'currentapprovalamount',
                    'payroll_proceed',
                    'dateapproved',
                    'jobsreported',
                    'staff_count',
                    'staff_salary_outlays']].rename(
    columns={'UNITID': 'ipeds', 'INSTNM': 'name'})

summary_table

,ipeds,name,ADDR,CITY,state,currentapprovalamount,payroll_proceed,dateapproved,jobsreported,staff_count,staff_salary_outlays
8,374024,Adler Graduate School,10225 Yellow Circle Dr.,Minnetonka,MN,600000.00,600000.00,[04/15/2020],32.0,50.0,3533616.0
9,142832,Adler University,17 North Dearborn Street,Chicago,IL,3956400.00,3956400.00,[04/14/2020],472.0,262.0,20951948.0
11,103501,Alaska Career College,1415 E Tudor Road,Anchorage,AK,456848.91,365479.14,[04/14/2020],49.0,35.0,1580730.0
12,442523,Alaska Christian College,35109 Royal Place,Soldotna,AK,386600.00,386600.00,[04/04/2020],46.0,13.0,810700.0
13,102669,Alaska Pacific University,4101 University Dr,Anchorage,AK,3421200.00,3421195.00,"[04/04/2020, 01/30/2021]",170.0,144.0,8180529.0
...,...,...,...,...,...,...,...,...,...,...,...
183,363934,University of Advancing Technology,2625 W Baseline Rd,Tempe,AZ,1231700.00,930000.00,['04/11/2020'],NaN,115.0,7501731.0
184,152336,University of Saint Francis-Fort Wayne,2701 Spring St,Fort Wayne,IN,5158000.00,5158000.00,['04/15/2020'],NaN,438.0,25953999.0
185,216542,University of Valley Forge,1401 Charlestown Road,Phoenixville,PA,1192400.00,1192400.00,['04/10/2020'],NaN,103.0,4914096.0
186,216807,Westminster College,319 South Market Street,New Wilmington,PA,3626920.00,2901536.00,['05/07/2020'],NaN,311.0,17180593.0


In [23]:
# Calculating ratios and pcts
summary_table['loan_staff_ratio'] = summary_table.currentapprovalamount / summary_table.staff_count
summary_table['loan_salary_pct'] = summary_table.currentapprovalamount / summary_table.staff_salary_outlays * 100

# Creating sorted tables
summary_table_sorted_salary = summary_table.sort_values(by='loan_salary_pct', ascending=False).reset_index(drop=True)
summary_table_sorted_staff = summary_table.sort_values(by='loan_staff_ratio', ascending=False).reset_index(drop=True)

print('SUMMARY TABLE SORTED BY LOAN SALARY PCT')
display(summary_table_sorted_salary.head(5))
print()
print('SUMMARY TABLE SORTED BY LOAN STAFF RATIO')
display(summary_table_sorted_staff.tail(5))

SUMMARY TABLE SORTED BY LOAN SALARY PCT


,ipeds,name,ADDR,CITY,state,currentapprovalamount,payroll_proceed,dateapproved,jobsreported,staff_count,staff_salary_outlays,loan_staff_ratio,loan_salary_pct
0,212993,Hussian College-Philadelphia,1500 Spring Garden Street Ste. 101,Philadelphia,PA,2532500.0,2532500.0,['04/07/2020'],NaN,11.0,616050.0,230227.272727,411.086762
1,456977,College of Business and Technology-Cutler Bay,19151 South Dixie Hwy # 205,Cutler Bay,FL,703010.0,703010.0,['04/28/2020'],NaN,13.0,450000.0,54077.692308,156.224444
2,134510,Hobe Sound Bible College,11298 SE Gomez Ave,Hobe Sound,FL,646850.0,646847.0,"[01/26/2021, 04/15/2020]",80.0,35.0,696292.0,18481.428571,92.899243
3,148876,Saint Augustine College,1345 W. Argyle St.,Chicago,IL,3153965.0,3153960.0,"['04/27/2020', '01/20/2021']",NaN,93.0,3866038.0,33913.602151,81.581324
4,193052,Mesivta Torah Vodaath Rabbinical Seminary,425 E 9th St,Brooklyn,NY,454552.5,454552.5,[04/27/2020],106.0,22.0,563960.0,20661.477273,80.600131



SUMMARY TABLE SORTED BY LOAN STAFF RATIO


,ipeds,name,ADDR,CITY,state,currentapprovalamount,payroll_proceed,dateapproved,jobsreported,staff_count,staff_salary_outlays,loan_staff_ratio,loan_salary_pct
715,213020,Indiana University of Pennsylvania-Main Campus,1011 South Dr 201 Sutton Hall,Indiana,PA,167387.0,167387.0,['04/13/2020'],NaN,1780.0,1.382954e+08,94.037640,0.121036
716,155399,Kansas State University,"919 Mid-Campus Drive, Anderson Hall",Manhattan,KS,410700.0,410700.0,['04/09/2020'],NaN,5436.0,3.816277e+08,75.551876,0.107618
717,234076,University of Virginia-Main Campus,1827 University Avenue,Charlottesville,VA,619600.0,619600.0,['04/08/2020'],NaN,8536.0,8.366613e+08,72.586692,0.074056
718,157289,University of Louisville,2301 S 3rd St,Louisville,KY,270500.0,270500.0,[04/15/2020],13.0,4509.0,3.424189e+08,59.991129,0.078997
719,243744,Stanford University,,Stanford,CA,253175.0,253175.0,['05/01/2020'],NaN,10811.0,1.425111e+09,23.418278,0.017765


In [24]:
# Calculating medians
loan_salary_pct_median = round(summary_table.loan_salary_pct.median(), 2)
loan_staff_ratio_median = round(summary_table.loan_staff_ratio.median(), 2)

print('LOAN SALARY PCT MEDIAN:')
display(loan_salary_pct_median)
print()
print('LOAN STAFF RATIO MEDIAN:')
display(loan_staff_ratio_median)

LOAN SALARY PCT MEDIAN:


20.41


LOAN STAFF RATIO MEDIAN:


11464.32

##### 6. EXPORTING CSVS

In [25]:
ipeds_loans_cp.to_csv('merged_loans_ipeds.csv')
summary_table.to_csv('merged_loans_ipeds_summary.csv')